In [3]:
import numpy as np
import pandas as pd
import scipy.stats as stats 
import statsmodels.stats.api as sms 
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
effect_size=sms.proportion_effectsize(0.13,0.15) # 根据预期比率计算效果量，0.13就是旧版转化率，0.15是新版转化率
effect_size

-0.0576728617308947

计算所需最小样本量

In [5]:
require_n=sms.NormalIndPower().solve_power(effect_size,alpha=0.05,power=0.8,ratio=1) # power为功效

In [6]:
require_n

4719.4740575998185

In [7]:
np.ceil(require_n) # 向上取整

4720.0

计算实验周期
根据上面最小样本量计算。得知此次AB测试至少需要9440用户参加，加入该落地页以往每天的平均浏览量为1000，则试验周期至少需要9440/1000=10天    

实例

In [8]:
data=pd.read_csv(r"C:\data\ab_data.csv")

In [9]:
data.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [12]:
data.dtypes

user_id          int64
timestamp       object
group           object
landing_page    object
converted        int64
dtype: object

In [21]:
import re

In [22]:
data['timestamp']=data['timestamp'].apply(lambda x:x.split(' ')[0])

In [24]:
data.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21,control,old_page,0
1,804228,2017-01-12,control,old_page,0
2,661590,2017-01-11,treatment,new_page,0
3,853541,2017-01-08,treatment,new_page,0
4,864975,2017-01-21,control,old_page,1


In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 7.9+ MB
